## LLM Pretraining

<div class="alert alert-block alert-success">
- This code demonstrates a toy example of how LLM pretraining works in a very simplified way.
- Your task is to complete the empty cells and fill the missing parts of the code indicated using the ellipsis "..."

Through this exercise you will learn:
- What a vocabulary construction looks like for a given dataset
- How tokenization can be done
- Encoding the data before training and decoding the data after generation
- Loss function most commonly used
- Making a forward pass
- Training elements like optimizers </div>

<div class="alert alert-block alert-warning">
Below demostrates a toy example that takes a text and used each character in the text as a "token". The code even if it works for you will probably not generate anything legible. The goal is that you should understand each element of the pretraining process. In reality, the training is a lot more sophisticated for many reasons - some being scale of the datasets, size of the models etc. The fundamentals on which these models are trained, however, can be demonstrated using this toy example. </div>

In [ ]:
!pip install torch

In [ ]:
# read file llm_pretraining.txt


In [ ]:
# create vocabulary, make sure the vocabulary is stored in a variable called "chars" (used later)


In [ ]:
# Tokenize
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
encoding_example = encode("hii there")
encoding_example

In [ ]:
decode(encoding_example)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# encode the dataset, make sure that after encoding the output is stored in a variable named "data" (used later)


In [ ]:
# training and test data split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[:n]

batch_size = 4
block_size = 8

In [ ]:
# batching function
# analyse this and see if you able to understand what is going on
def get_batch(split):
    data_ = train_data if split == "train" else val_data
    ix = torch.randint(len(data_) - block_size, (batch_size,))
    x = torch.stack([data_[i:i+block_size] for i in ix])
    y = torch.stack([data_[i+1:i+block_size+1] for i in ix])
    return x, y


xb, yb = get_batch('train')
print(yb)

# bigram models with the appropriate functions
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size_):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size_, vocab_size_)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = ... # cross entropy loss function

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
# initialize the BigramLanguageModel, make sure the variable name is "m" (used later)
# make a forward pass with the BigramLanguageModel Class, this step is just a test to check if the call works and not part of the training


In [ ]:
# training
optimizer = ... # initialise the optimiser
batch_size = ... # set a batch size
for steps in range(1000):  # 1000 steps training, you may change this
    xb, yb = ... # get the training batch

    logits, loss =  # make the forward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()  # backpropagation
    optimizer.step()
print(loss.item())

In [ ]:
# this step generates text using your trained model
# it probably won't generate anything interesting (or maybe it will?)
# by the time you reach this step you should have uncerstood the principals of:
# 1. How the pre-training works
# 2. Can go back to the corret sources to understand how to expand on this basic knowledge

input_data = torch.tensor([encode("Let us kill him")], dtype=torch.long)  # you can change the encoding sentence to something else
generate_n_tokens = 500 # you can change max_new_tokens=500 to change size of the generation
print(decode(m.generate(idx=input_data, max_new_tokens=generate_n_tokens)[0].tolist()))